<img src="images/econ140R_logo.png" width="200" />

<h1>ECON 140R Class 19</h1>

In the long-long ago of Fall 1993, I completed the equivalent of ECON 140 at college. The first term was led that year by Henry Farber, while Orley Ashenfelter taught econometric applications for undergraduates in Spring 1994, which is roughly equivalent to ECON c142. At Berkeley, David Card led ECON c142 before his retirement from teaching after Spring 2022. Typically, Farber taught the second term while Ashenfelter taught the first term, but in this year they had switched.

From this period, I most remember Prof. Farber's pro-union remarks, and his exhortations to struggle a little while learning Stata and econometrics. I also distinctly remember Prof. Ashenfelter's wine regressions, which were later published as [Orley Ashenfelter, David Ashmore, and Robert Lalonde (1995) "Bordeaux Wine Vintage Quality and the Weather," *Chance* 8(4): 7-14.](https://www-tandfonline-com.libproxy.berkeley.edu/doi/abs/10.1080/09332480.1995.10542468)

As of late October 2023, a writeup and the data were still available at http://www.liquidasset.com/orley.htm and http://www.liquidasset.com/winedata.html.

Learning objectives:

1. When you have time-series data, literally looking at the series over time is helpful
2. Weather data are superb $x$-variables
3. Sometimes the best projects are fun, like examining how wine critics are sometimes silly and incorrect
4. Bordeaux wine, like California cabernet sauvignon and similar varietals, is better with age, wet winters, hot  growing seasons, and dry harvests. Rain before the harvest is bad 

Here are the load calls for time-series nifty moves, which we saw in Class 18. 

In [ ]:
library(haven)
install.packages("dynlm")
library(dynlm)
library(lmtest)
install.packages("orcutt")
library(orcutt)

Here are the data. These are mostly the data you can see at http://www.liquidasset.com/winedata.html at least as of late October 2023. One difference is that I rescaled `price` to show a $407 per-bottle average price as of 1990-1991 of the 1961 vintage, which is reported in Table 1 of [Ashenfelter, Ashmore, and Lalonde (1995)](https://www-tandfonline-com.libproxy.berkeley.edu/doi/abs/10.1080/09332480.1995.10542468). The log of the price, `lprice`, is also the log of that value. The only thing this changes is the constant term.

In [ ]:
ashenfelter <- read_dta("data/ashenfelter_chance95.dta")
head(ashenfelter)

<h2>Bordeaux wine prices in 1990/91 and historical weather</h2>

Ashenfelter was driven by a desire to shed light on the quality of a Bordeaux vintage that was unattached to the "expert opinion" of wine critics, of whom he did not think highly. The dataset is a mix of 

1. cross-sectional data on the market prices in 1990 and 1991 of different vintages and vineyards (chateaux), as published in *Liquid Assets* in 1991 and 1992

2. time-series historical data on weather in Bordeaux for the vintages

The data descriptions are:

* `vint` is the year of the vintage 
* `time_sv` is the years between the vintage and the year 1990
* `wrain` is mL of winter rain (October to March)
* `hrain` is mL of harvest rain (August and September)
* `degrees` is average temperature in degrees Centigrade between April and September 

The __R__ package `dynlm` contains some special tools for dealing with time series, and `ts()` is one of them. Florian Heiss uses this package and these data in [Section 12.1](http://www.urfie.net/read/index.html#page/216).

A challenge is that there are no price measures for the 1954 and 1956 vintages, which were considered to be the poorest in their decade according to [Ashenfelter, Ashmore, and Lalonde (1995)](https://www-tandfonline-com.libproxy.berkeley.edu/doi/abs/10.1080/09332480.1995.10542468). __R__ does not like missing data. :)

In [ ]:
tsdata <- ts(ashenfelter, start = 1952, end = 1989)
head(tsdata)

<h2>Visualization</h2>

One cut is to look at the obvious choice of the $y$-variable, the log price, as a function of time.

In [ ]:
plot(ashenfelter$vint, ashenfelter$lprice)

We see a downward sloping relationship here, as any oenophile would tell you: the price of a wine that ages well, like a Bordeaux, typically rises with age. Here, the vintage is along the $x$-axis, so points further left are prices of wines that have aged more.

<h2>OLS results</h2>

Our first approach is simple OLS, here using `dynlm()` for no particular reason. We could also use `lm()` and get the same results.

Here is a very simple model, showing log price regressed only on time since vintage:

In [ ]:
reg_ts0 <- dynlm(lprice ~ time_sv,
                 data = tsdata)
summary(reg_ts0)

Recall that the price data are retrieved from a moment in time; these are not prices at first bottling or anything historical. They measure relative prices at a point in time, and thus inflation is not a factor.

<font color="blue">Interpret the coefficient on years of time ("time_sv") here in terms of a familiar financial parameter.</font>

<hr>

Before we run the full model, it might be useful to examine the time series characteristics of the $x$-variables we will consider. Winter rain, harvest rain, and growing-season temperature are in the dataset; but what do they look like in time series?

Do we see trending variables, which might produce spurious correlations because the log price is also trending?

In [ ]:
plot(ashenfelter$vint, ashenfelter$degrees)

In [ ]:
plot(ashenfelter$vint, ashenfelter$hrain)

In [ ]:
plot(ashenfelter$vint, ashenfelter$wrain)

Unlike most time series you will see, these show no trends at all before 1990. Whether they might have later is unclear; there have been more than 30 years of climate change since the data were collected, and it is certainly possible.

But for now, these $x$-variables show *no clear time trend*, which is a good thing for understanding how $y$ actually varies with $x$'s using traditional tools.

Let us run the regression with each of these 3 additional $x$ variables:

In [ ]:
reg_ts <- dynlm(lprice ~ time_sv 
                       + degrees 
                       + hrain 
                       + wrain, 
                       data = tsdata)
summary(reg_ts)

<font color="blue">Interpret the coefficient on years of time ("time_sv") here in terms of a familiar financial parameter. What has happened to it?</font>

As a check, let us look at the residuals of the regression over time. A potential issue is that __R__ does not like time series with missing values, but calling `plot()` on the residuals dataframe appears to work OK.

In [ ]:
plot(reg_ts$resid)
grid()

I think these residuals look pretty well behaved. They are not consistently tracking above or below zero. But the DW statistic can help us see things that our eyes cannot.

<h2>Durbin-Watson statistic</h2>

Recall the [Durbin-Watson statistic](https://en.wikipedia.org/wiki/Durbin%E2%80%93Watson_statistic), here calculated by calling `dwtest()` on the OLS regression that we just ran. The DW statistic is a standard diagnostic measure in time series analysis. If the DW stat is near the number 2, there is little evidence of serially correlated errors. In large samples, the DW stat trends toward $d = 2(1 - \rho)$ where $\rho$ is the first-order autocorrelation in the residuals:

$$
\epsilon_t = \rho \ \epsilon_{t-1} + \nu_t
$$

Thus if there is positive serial correlation, $\rho \in (0,1)$, and the DW stat is less than 2. If there is negative serial correlation, $\rho \in (-1,0)$, and the statistic is greater than 2. (We cannot have $| \rho | > 1$, because if that were the case, the errors would grow forever.)

<h4>What level of the DW statistic is reasonable?</h4> I am not certain whether there are hard-and-fast rules. The range of 1.5 - 2.5 appears to be the standard "acceptable range" in the literature and on various websites.

In [ ]:
dwtest(reg_ts)

We have some *negative* serial correlation in the OLS model, but the DW stat of 2.8 is not far outside normal range.

<hr>

<div style="text-align: right"> <span style="font-family:Papyrus; ">And they lived happily ever after. The End.</span></div>